In [1]:
import os
import pandas as pd
from collections import defaultdict
import nibabel as nib
from glob import glob
import subprocess
import sys
from scipy.io import savemat
import numpy as np
import gc
import pickle
from numpy import savez_compressed
import gzip, pickle, pickletools
import glob
import re
import nibabel as nib
import torch

In [2]:
# Manual one-hot encoding
onehot = {'imagenet': [1,0,0,0],
          'rep_imagenet': [1,0,0,0],
          'coco': [0,1,0,0],
          'rep_coco': [0,1,0,0],
          'scenes': [0,0,1,0],
          'rep_scenes': [0,0,1,0],
          'none': [0,0,0,1]}

In [3]:
#Change subject name here
subname = "CSI2"

preproc_dir = "/bigpool/export/users/datasets_faprak2020/BOLD5000/preprocessed/"
preproc_files = glob.glob(preproc_dir+ 'sub-'+ subname + '*_preproc.nii.gz')
mask_dir = '/bigpool/export/users/datasets_faprak2020/BOLD5000/preprocessed/masks/'
data_dir = ("/bigpool/export/users/datasets_faprak2020/BOLD5000/traindata_cnn/{}/".format(subname))
file_list = glob.glob(mask_dir + 'sub-{}_mask-*-resamp.nii.gz'.format(subname))
events_dir = "/bigpool/export/users/datasets_faprak2020/BOLD5000/ds001499-download/"

subjects = ['CSI1', 'CSI2','CSI3','CSI4']


In [4]:
super_categories_path = '/home/joshini/image_labels.tsv'
with open(super_categories_path,'r') as f:
    d = pd.read_csv(f, delimiter = r'\t',engine='python',names = ['ImgName', 'ClusterNum', 'Category'] )

In [5]:
d.loc[4912] = ['fireplace2.jpg', '9','indoor spaces']
d.loc[4913] = ['fireplace3.jpg', '9','indoor spaces']
d.loc[4914] = ['fireplace4.jpg', '9','indoor spaces']
d.loc[4915] = ['fireplace5.jpg', '9','indoor spaces']

In [6]:
d.head()

,ImgName,ClusterNum,Category
0,ATM1.jpg,9,indoor spaces
1,ATM2.jpg,9,indoor spaces
2,ATM4.jpg,3,line patterned
3,ATM5.jpg,9,indoor spaces
4,COCO_train2014_000000000036.jpg,2,people


In [7]:
d.set_index("ImgName", inplace=True)

In [8]:
file_list

['/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-LHEarlyVis-resamp.nii.gz',
 '/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-RHEarlyVis-resamp.nii.gz',
 '/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-RHPPA-resamp.nii.gz',
 '/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-RHOPA-resamp.nii.gz',
 '/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-RHLO-resamp.nii.gz',
 '/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-LHLO-resamp.nii.gz',
 '/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-LHOPA-resamp.nii.gz',
 '/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-LHRSC-resamp.nii.gz',
 '/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-LHPPA-resamp.nii.gz',
 '/home/joshini/simlink_dataset/preprocessed/masks/sub-CSI2_mask-RHRSC-resamp.nii.gz']

In [9]:
# Checking shape of CSI masks. All shapes should be the same.
for f in file_list:
    print(nib.load(f).get_fdata().shape)
    

(91, 109, 91)
(91, 109, 91)
(91, 109, 91)
(91, 109, 91)
(91, 109, 91)
(91, 109, 91)
(91, 109, 91)
(91, 109, 91)
(91, 109, 91)
(91, 109, 91)


In [10]:
#Combining all masks
final_mask = np.zeros((91, 109, 91),dtype=np.float32)
for i,f in enumerate(file_list):
    final_mask += nib.load(f).get_fdata()
print(final_mask.shape)

(91, 109, 91)


In [11]:
np.unique(final_mask)

array([0.   , 0.125, 0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   ,
       1.125, 1.25 , 1.375, 1.5  , 1.625, 1.75 , 1.875, 2.   ],
      dtype=float32)

In [12]:
final_mask[np.where(final_mask<0.5)]=0
final_mask[np.where(final_mask>=0.5)]=1

In [13]:
np.unique(final_mask)

array([0., 1.], dtype=float32)

In [14]:
for pnum, preproc in enumerate(preproc_files):
    X,Y,Ynames,Ycluster = [], [], [],[]
    print('\t\tPreprocessed file %d out of %d' % ((pnum + 1), len(preproc_files)))
    mask = final_mask
    
    items = preproc.split('_')
    ses = items[-3]
    run = items[-2]
    
    event_file = glob.glob(os.path.join(events_dir,'sub-CSI2',ses,'func','*' + run + '_events.tsv'))[0]
    # Load events and image
    events = pd.read_csv(event_file, sep = '\t')

    img_proxy = nib.load(preproc)
    
    # Apply mask
    start_all, end_all = [], []
    for index, row in events.iterrows():
        # Beginning TR of trial
        start = int(round(row['onset']) / 2)
        start_all.append(start)
        
        # Ending TR of trial, start + 10 sec, or 5 TRs
        end = start + 5
        end_all.append(end)
    
    img = img_proxy.dataobj[:,:,:,start_all[0]:end_all[-1]]
    
    img = np.transpose(img, axes=(3,0,1,2))
    roi = np.where(mask==0, 0, img)
    print("ROI") # Shape: voxels x TRs
    print(roi.shape)
    assert img.shape == roi.shape
    img_proxy.uncache()
    del img_proxy
    del img
    gc.collect()
    for index, row in events.iterrows():
        # Beginning TR of trial
        start = int(round(row['onset']) / 2)
        # Ending TR of trial, start + 10 sec, or 5 TRs
        end = start + 5
        if not end > roi.shape[0]:
            x = roi[start:end,:,:,:]#.T
            y = int(d.loc[row['ImgName'],['ClusterNum']])
            print(x.shape)
            print(y)
            dictionary = {'X':x, 'y':y}
            torch.save(dictionary, data_dir + subname + '_' + str(pnum) + '_' + str(index)+'.p')
            
    # Save last ten TRs as no stimulus, if enough data is left
    if roi.shape[0] - end >= 5:
        x = roi[end:end+5,:,:,:]#.T
        if x.shape[1] != 5: break
        print('End')
        print(end,end+5)
        y = 10
        print(x.shape)
        dictionary = {'X':x, 'y':y}
        torch.save(dictionary, data_dir + subname + '_' + str(pnum) + '_' + str(index)+'.p')

		Preprocessed file 1 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
9
(5, 91, 109, 91)
7
(5, 91, 109, 91)
1
(5, 91, 109, 91)
0
(5, 91, 109, 91)
5
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
3
(5, 91, 109, 91)
8
(5, 91, 109, 91)
8
(5, 91, 109, 91)
6
(5, 91, 109, 91)
7
(5, 91, 109, 91)
3
(5, 91, 109, 91)
8
(5, 91, 109, 91)
4
(5, 91, 109, 91)
1
(5, 91, 109, 91)
0
(5, 91, 109, 91)
9
(5, 91, 109, 91)
1
(5, 91, 109, 91)
6
(5, 91, 109, 91)
5
(5, 91, 109, 91)
6
(5, 91, 109, 91)
8
(5, 91, 109, 91)
2
(5, 91, 109, 91)
6
(5, 91, 109, 91)
2
(5, 91, 109, 91)
0
(5, 91, 109, 91)
8
(5, 91, 109, 91)
7
(5, 91, 109, 91)
4
(5, 91, 109, 91)
8
(5, 91, 109, 91)
5
(5, 91, 109, 91)
2
(5, 91, 109, 91)
2
(5, 91, 109, 91)
5
(5, 91, 109, 91)
6
		Preprocessed file 2 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
5
(5, 91, 109, 91)
9
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
6
(5, 91, 109, 91)
5
(5, 91, 109, 91)
1
(5, 91, 109, 91)
4
(5, 91, 109, 91)
5
(5, 91, 109, 9

(5, 91, 109, 91)
4
(5, 91, 109, 91)
5
(5, 91, 109, 91)
5
(5, 91, 109, 91)
7
(5, 91, 109, 91)
0
(5, 91, 109, 91)
3
(5, 91, 109, 91)
2
(5, 91, 109, 91)
6
(5, 91, 109, 91)
7
(5, 91, 109, 91)
7
(5, 91, 109, 91)
2
(5, 91, 109, 91)
0
(5, 91, 109, 91)
4
(5, 91, 109, 91)
8
(5, 91, 109, 91)
1
(5, 91, 109, 91)
1
(5, 91, 109, 91)
8
(5, 91, 109, 91)
5
(5, 91, 109, 91)
8
(5, 91, 109, 91)
2
(5, 91, 109, 91)
2
(5, 91, 109, 91)
4
(5, 91, 109, 91)
3
(5, 91, 109, 91)
7
(5, 91, 109, 91)
4
(5, 91, 109, 91)
3
(5, 91, 109, 91)
1
(5, 91, 109, 91)
7
(5, 91, 109, 91)
6
(5, 91, 109, 91)
6
(5, 91, 109, 91)
2
(5, 91, 109, 91)
6
(5, 91, 109, 91)
3
(5, 91, 109, 91)
3
(5, 91, 109, 91)
6
		Preprocessed file 13 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
9
(5, 91, 109, 91)
0
(5, 91, 109, 91)
4
(5, 91, 109, 91)
4
(5, 91, 109, 91)
2
(5, 91, 109, 91)
8
(5, 91, 109, 91)
0
(5, 91, 109, 91)
9
(5, 91, 109, 91)
4
(5, 91, 109, 91)
5
(5, 91, 109, 91)
3
(5, 91, 109, 91)
3
(5, 91, 109, 91)
4
(5, 91, 109, 91)
8
(5, 91, 109,

(5, 91, 109, 91)
4
(5, 91, 109, 91)
6
(5, 91, 109, 91)
1
(5, 91, 109, 91)
5
(5, 91, 109, 91)
3
(5, 91, 109, 91)
6
(5, 91, 109, 91)
3
(5, 91, 109, 91)
3
(5, 91, 109, 91)
1
(5, 91, 109, 91)
0
(5, 91, 109, 91)
9
(5, 91, 109, 91)
9
(5, 91, 109, 91)
5
(5, 91, 109, 91)
1
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
2
(5, 91, 109, 91)
0
(5, 91, 109, 91)
4
(5, 91, 109, 91)
8
(5, 91, 109, 91)
3
(5, 91, 109, 91)
3
(5, 91, 109, 91)
8
(5, 91, 109, 91)
2
(5, 91, 109, 91)
4
(5, 91, 109, 91)
2
(5, 91, 109, 91)
0
(5, 91, 109, 91)
3
(5, 91, 109, 91)
5
(5, 91, 109, 91)
4
(5, 91, 109, 91)
8
(5, 91, 109, 91)
0
		Preprocessed file 24 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
4
(5, 91, 109, 91)
6
(5, 91, 109, 91)
5
(5, 91, 109, 91)
6
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
7
(5, 91, 109, 91)
2
(5, 91, 109, 91)
8
(5, 91, 109, 91)
4
(5, 91, 109, 91)
9
(5, 91, 109, 91)
3
(5, 91, 109, 91)
0
(5, 91, 109, 91)
6
(5, 91, 109, 91)
2
(5, 91, 109, 91)
3
(5, 91, 109, 91)
8
(5, 91, 109,

(5, 91, 109, 91)
4
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
8
(5, 91, 109, 91)
2
(5, 91, 109, 91)
0
(5, 91, 109, 91)
5
(5, 91, 109, 91)
8
(5, 91, 109, 91)
7
(5, 91, 109, 91)
2
(5, 91, 109, 91)
7
(5, 91, 109, 91)
0
(5, 91, 109, 91)
2
(5, 91, 109, 91)
7
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
5
(5, 91, 109, 91)
1
(5, 91, 109, 91)
4
(5, 91, 109, 91)
9
(5, 91, 109, 91)
8
(5, 91, 109, 91)
5
(5, 91, 109, 91)
8
(5, 91, 109, 91)
0
(5, 91, 109, 91)
4
(5, 91, 109, 91)
2
(5, 91, 109, 91)
1
(5, 91, 109, 91)
7
(5, 91, 109, 91)
9
		Preprocessed file 35 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
1
(5, 91, 109, 91)
4
(5, 91, 109, 91)
4
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
5
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
3
(5, 91, 109, 91)
7
(5, 91, 109, 91)
4
(5, 91, 109, 91)
0
(5, 91, 109, 91)
9
(5, 91, 109, 91)
8
(5, 91, 109, 91)
7
(5, 91, 109, 91)
2
(5, 91, 109, 91)
2
(5, 91, 109, 91)
4
(5, 91, 109, 91)
0
(5, 91, 109, 91)
2
(5, 91, 109,

(5, 91, 109, 91)
2
(5, 91, 109, 91)
0
(5, 91, 109, 91)
2
(5, 91, 109, 91)
3
(5, 91, 109, 91)
2
(5, 91, 109, 91)
9
(5, 91, 109, 91)
2
(5, 91, 109, 91)
3
(5, 91, 109, 91)
6
(5, 91, 109, 91)
1
(5, 91, 109, 91)
4
(5, 91, 109, 91)
9
(5, 91, 109, 91)
0
(5, 91, 109, 91)
9
(5, 91, 109, 91)
7
(5, 91, 109, 91)
9
(5, 91, 109, 91)
6
(5, 91, 109, 91)
4
(5, 91, 109, 91)
3
(5, 91, 109, 91)
2
(5, 91, 109, 91)
0
(5, 91, 109, 91)
6
(5, 91, 109, 91)
6
(5, 91, 109, 91)
7
(5, 91, 109, 91)
1
(5, 91, 109, 91)
4
		Preprocessed file 46 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
3
(5, 91, 109, 91)
3
(5, 91, 109, 91)
7
(5, 91, 109, 91)
0
(5, 91, 109, 91)
8
(5, 91, 109, 91)
5
(5, 91, 109, 91)
0
(5, 91, 109, 91)
2
(5, 91, 109, 91)
5
(5, 91, 109, 91)
7
(5, 91, 109, 91)
4
(5, 91, 109, 91)
3
(5, 91, 109, 91)
6
(5, 91, 109, 91)
2
(5, 91, 109, 91)
9
(5, 91, 109, 91)
6
(5, 91, 109, 91)
9
(5, 91, 109, 91)
6
(5, 91, 109, 91)
8
(5, 91, 109, 91)
3
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
6
(5, 91, 109,

...

(5, 91, 109, 91)
1
(5, 91, 109, 91)
9
(5, 91, 109, 91)
0
(5, 91, 109, 91)
3
(5, 91, 109, 91)
6
(5, 91, 109, 91)
7
(5, 91, 109, 91)
1
(5, 91, 109, 91)
6
(5, 91, 109, 91)
4
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
6
(5, 91, 109, 91)
7
(5, 91, 109, 91)
9
		Preprocessed file 90 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
2
(5, 91, 109, 91)
0
(5, 91, 109, 91)
7
(5, 91, 109, 91)
0
(5, 91, 109, 91)
8
(5, 91, 109, 91)
3
(5, 91, 109, 91)
1
(5, 91, 109, 91)
4
(5, 91, 109, 91)
4
(5, 91, 109, 91)
3
(5, 91, 109, 91)
5
(5, 91, 109, 91)
5
(5, 91, 109, 91)
7
(5, 91, 109, 91)
5
(5, 91, 109, 91)
2
(5, 91, 109, 91)
3
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
2
(5, 91, 109, 91)
9
(5, 91, 109, 91)
6
(5, 91, 109, 91)
8
(5, 91, 109, 91)
5
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
7
(5, 91, 109, 91)
1
(5, 91, 109, 91)
5
(5, 91, 109, 91)
9
(5, 91, 109, 91)
5
(5, 91, 109, 91)
0
(5, 91, 109, 91)
2
(5, 91, 109, 91)
9
(5, 91, 109, 91)
2
(5, 91, 109, 91)
8
(5, 91, 109,

(5, 91, 109, 91)
6
(5, 91, 109, 91)
7
(5, 91, 109, 91)
1
(5, 91, 109, 91)
6
(5, 91, 109, 91)
3
(5, 91, 109, 91)
1
(5, 91, 109, 91)
9
(5, 91, 109, 91)
9
(5, 91, 109, 91)
1
(5, 91, 109, 91)
5
(5, 91, 109, 91)
6
		Preprocessed file 101 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
2
(5, 91, 109, 91)
5
(5, 91, 109, 91)
1
(5, 91, 109, 91)
5
(5, 91, 109, 91)
2
(5, 91, 109, 91)
4
(5, 91, 109, 91)
1
(5, 91, 109, 91)
5
(5, 91, 109, 91)
6
(5, 91, 109, 91)
2
(5, 91, 109, 91)
5
(5, 91, 109, 91)
7
(5, 91, 109, 91)
7
(5, 91, 109, 91)
8
(5, 91, 109, 91)
8
(5, 91, 109, 91)
9
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
3
(5, 91, 109, 91)
6
(5, 91, 109, 91)
8
(5, 91, 109, 91)
5
(5, 91, 109, 91)
4
(5, 91, 109, 91)
7
(5, 91, 109, 91)
6
(5, 91, 109, 91)
0
(5, 91, 109, 91)
9
(5, 91, 109, 91)
9
(5, 91, 109, 91)
0
(5, 91, 109, 91)
3
(5, 91, 109, 91)
6
(5, 91, 109, 91)
9
(5, 91, 109, 91)
5
(5, 91, 109, 91)
7
(5, 91, 109, 91)
0
(5, 91, 109, 91)
3
		Preprocessed file 102 out of 142
ROI
(185, 91, 

(5, 91, 109, 91)
4
(5, 91, 109, 91)
6
(5, 91, 109, 91)
0
(5, 91, 109, 91)
8
(5, 91, 109, 91)
5
(5, 91, 109, 91)
8
(5, 91, 109, 91)
3
(5, 91, 109, 91)
2
(5, 91, 109, 91)
7
		Preprocessed file 112 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
1
(5, 91, 109, 91)
8
(5, 91, 109, 91)
6
(5, 91, 109, 91)
9
(5, 91, 109, 91)
4
(5, 91, 109, 91)
8
(5, 91, 109, 91)
2
(5, 91, 109, 91)
6
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
2
(5, 91, 109, 91)
5
(5, 91, 109, 91)
9
(5, 91, 109, 91)
6
(5, 91, 109, 91)
1
(5, 91, 109, 91)
0
(5, 91, 109, 91)
4
(5, 91, 109, 91)
2
(5, 91, 109, 91)
9
(5, 91, 109, 91)
6
(5, 91, 109, 91)
0
(5, 91, 109, 91)
4
(5, 91, 109, 91)
3
(5, 91, 109, 91)
9
(5, 91, 109, 91)
2
(5, 91, 109, 91)
3
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
2
(5, 91, 109, 91)
2
(5, 91, 109, 91)
5
(5, 91, 109, 91)
1
(5, 91, 109, 91)
0
(5, 91, 109, 91)
9
(5, 91, 109, 91)
2
(5, 91, 109, 91)
5
		Preprocessed file 113 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
3
(5, 91, 10

(5, 91, 109, 91)
5
(5, 91, 109, 91)
0
(5, 91, 109, 91)
8
(5, 91, 109, 91)
5
(5, 91, 109, 91)
9
(5, 91, 109, 91)
0
(5, 91, 109, 91)
8
		Preprocessed file 123 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
1
(5, 91, 109, 91)
8
(5, 91, 109, 91)
9
(5, 91, 109, 91)
3
(5, 91, 109, 91)
4
(5, 91, 109, 91)
9
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
5
(5, 91, 109, 91)
8
(5, 91, 109, 91)
6
(5, 91, 109, 91)
9
(5, 91, 109, 91)
3
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
3
(5, 91, 109, 91)
2
(5, 91, 109, 91)
5
(5, 91, 109, 91)
4
(5, 91, 109, 91)
4
(5, 91, 109, 91)
6
(5, 91, 109, 91)
0
(5, 91, 109, 91)
4
(5, 91, 109, 91)
5
(5, 91, 109, 91)
6
(5, 91, 109, 91)
4
(5, 91, 109, 91)
5
(5, 91, 109, 91)
8
(5, 91, 109, 91)
7
(5, 91, 109, 91)
0
(5, 91, 109, 91)
6
(5, 91, 109, 91)
2
(5, 91, 109, 91)
1
(5, 91, 109, 91)
9
(5, 91, 109, 91)
2
		Preprocessed file 124 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
9
(5, 91, 109, 91)
9
(5, 91, 109, 91)
3
(5, 91, 10

(5, 91, 109, 91)
7
(5, 91, 109, 91)
5
(5, 91, 109, 91)
4
(5, 91, 109, 91)
4
(5, 91, 109, 91)
7
		Preprocessed file 134 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
0
(5, 91, 109, 91)
0
(5, 91, 109, 91)
1
(5, 91, 109, 91)
6
(5, 91, 109, 91)
8
(5, 91, 109, 91)
4
(5, 91, 109, 91)
2
(5, 91, 109, 91)
0
(5, 91, 109, 91)
6
(5, 91, 109, 91)
3
(5, 91, 109, 91)
2
(5, 91, 109, 91)
5
(5, 91, 109, 91)
5
(5, 91, 109, 91)
0
(5, 91, 109, 91)
8
(5, 91, 109, 91)
0
(5, 91, 109, 91)
6
(5, 91, 109, 91)
0
(5, 91, 109, 91)
4
(5, 91, 109, 91)
2
(5, 91, 109, 91)
8
(5, 91, 109, 91)
8
(5, 91, 109, 91)
1
(5, 91, 109, 91)
5
(5, 91, 109, 91)
8
(5, 91, 109, 91)
3
(5, 91, 109, 91)
0
(5, 91, 109, 91)
5
(5, 91, 109, 91)
1
(5, 91, 109, 91)
9
(5, 91, 109, 91)
3
(5, 91, 109, 91)
1
(5, 91, 109, 91)
2
(5, 91, 109, 91)
1
(5, 91, 109, 91)
0
(5, 91, 109, 91)
3
		Preprocessed file 135 out of 142
ROI
(185, 91, 109, 91)
(5, 91, 109, 91)
8
(5, 91, 109, 91)
9
(5, 91, 109, 91)
1
(5, 91, 109, 91)
9
(5, 91, 109, 91)
6
(5, 91, 10